In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge,RidgeCV#线性回归模型的岭回归
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier     #随机森林用于分类
from sklearn.ensemble import RandomForestRegressor as RFR      #随机森林用于回归
from sklearn import metrics    
from statsmodels.formula.api import ols


In [5]:
polymerdata = pd.read_excel('polymer-dragon-all.xlsx')
polymerdata.head()


(130, 4887)


,NAME,PCE(%),MW,AMW,Sv,Se,Sp,Si,Mv,Me,...,Psychotic-80,Psychotic-50,Hypertens-80,Hypertens-50,Hypnotic-80,Hypnotic-50,Neoplastic-80,Neoplastic-50,Infective-80,Infective-50
0,BHJ-P1-iPr,10.90,2756.80,7.105,239.102,378.979,269.094,430.489,0.616,0.977,...,0,0,0,0,0,0,0,0,0,0
1,BHJ-P1-Me,15.66,2700.68,7.183,232.995,367.444,262.048,416.829,0.620,0.977,...,0,0,0,0,0,0,0,0,0,0
2,D16,16.22,2602.66,7.230,222.843,350.972,252.901,398.423,0.619,0.975,...,0,0,0,0,0,0,0,0,0,0
3,D18,17.60,2722.68,7.480,227.979,357.008,256.242,404.526,0.626,0.981,...,0,0,0,0,0,0,0,0,0,0
4,D18-Cl,16.74,2788.48,7.661,229.756,356.252,259.923,402.943,0.631,0.979,...,0,0,0,0,0,0,0,0,0,0


In [28]:
polymerdata = polymerdata.rename(columns={'PCE(%)':'PCE'})

In [29]:
# 删除0值和空值
polymerdata  = polymerdata.dropna(axis=1,how='any')
data0=(polymerdata==0).sum(axis = 0)
number0 = data0[data0>=60]
number0.index
polymerdata = polymerdata.drop(labels=number0.index,axis=1)
print(polymerdata.shape)
polymerdata.head()

(130, 2646)


,NAME,PCE,MW,AMW,Sv,Se,Sp,Si,Mv,Me,...,DLS_01,DLS_02,DLS_03,DLS_04,DLS_05,DLS_06,DLS_07,DLS_cons,LLS_01,LLS_02
0,BHJ-P1-iPr,10.90,2756.80,7.105,239.102,378.979,269.094,430.489,0.616,0.977,...,0.5,0.33,0.5,0.6,0.5,0.33,0.5,0.47,0.17,0.5
1,BHJ-P1-Me,15.66,2700.68,7.183,232.995,367.444,262.048,416.829,0.620,0.977,...,0.5,0.33,0.5,0.6,0.5,0.33,0.5,0.47,0.17,0.5
2,D16,16.22,2602.66,7.230,222.843,350.972,252.901,398.423,0.619,0.975,...,0.5,0.33,0.5,0.6,0.5,0.33,0.5,0.47,0.33,0.5
3,D18,17.60,2722.68,7.480,227.979,357.008,256.242,404.526,0.626,0.981,...,0.5,0.33,0.5,0.6,0.5,0.33,0.5,0.47,0.17,0.5
4,D18-Cl,16.74,2788.48,7.661,229.756,356.252,259.923,402.943,0.631,0.979,...,0.5,0.33,0.5,0.6,0.5,0.33,0.5,0.47,0.17,0.5


In [30]:
# 删除低方差的值
var_data = polymerdata.iloc[:,2:].var()
del_col = var_data[var_data<0.1].index
polymerdata = polymerdata.drop(labels=del_col,axis=1)
print(polymerdata.shape)
polymerdata.head()

(130, 1513)


,NAME,PCE,MW,AMW,Sv,Se,Sp,Si,nAT,nSK,...,MLOGP,MLOGP2,ALOGP,ALOGP2,Vx,VvdwMG,VvdwZAZ,BLTF96,BLTD48,BLTA96
0,BHJ-P1-iPr,10.90,2756.80,7.105,239.102,378.979,269.094,430.489,388,184,...,19.723,389.009,62.066,3852.213,3633.821,1492.648,2870.56,-18.15,-20.06,-20.95
1,BHJ-P1-Me,15.66,2700.68,7.183,232.995,367.444,262.048,416.829,376,180,...,19.331,373.696,60.614,3674.009,3540.199,1454.278,2801.36,-17.82,-19.68,-20.56
2,D16,16.22,2602.66,7.230,222.843,350.972,252.901,398.423,360,170,...,20.338,413.648,62.272,3877.852,3419.269,1404.717,2688.08,-18.68,-20.64,-21.57
3,D18,17.60,2722.68,7.480,227.979,357.008,256.242,404.526,364,178,...,20.606,424.618,63.627,4048.370,3474.219,1427.237,2750.98,-18.91,-20.90,-21.84
4,D18-Cl,16.74,2788.48,7.661,229.756,356.252,259.923,402.943,364,178,...,20.805,432.840,65.462,4285.326,3543.854,1455.776,2787.54,-19.07,-21.08,-22.03


In [31]:
# 删除重复值多的列
Duplicated_series=pd.Series(np.zeros(len(polymerdata.columns)))
Duplicated_series.index = polymerdata.columns.values
for i in polymerdata.columns:
    Duplicated_series[i] = len(Counter(polymerdata[i]))
    
Duplicated_series = Duplicated_series.sort_values(0)    
# Duplicated_series60 = Duplicated_series[Duplicated_series>60]    
Duplicated_series15 = Duplicated_series[Duplicated_series<=15]
polymerdata = polymerdata.drop(labels=Duplicated_series15.index,axis=1)
print(polymerdata.shape)
polymerdata.head()

(130, 1437)


,NAME,PCE,MW,AMW,Sv,Se,Sp,Si,nAT,nSK,...,MLOGP,MLOGP2,ALOGP,ALOGP2,Vx,VvdwMG,VvdwZAZ,BLTF96,BLTD48,BLTA96
0,BHJ-P1-iPr,10.90,2756.80,7.105,239.102,378.979,269.094,430.489,388,184,...,19.723,389.009,62.066,3852.213,3633.821,1492.648,2870.56,-18.15,-20.06,-20.95
1,BHJ-P1-Me,15.66,2700.68,7.183,232.995,367.444,262.048,416.829,376,180,...,19.331,373.696,60.614,3674.009,3540.199,1454.278,2801.36,-17.82,-19.68,-20.56
2,D16,16.22,2602.66,7.230,222.843,350.972,252.901,398.423,360,170,...,20.338,413.648,62.272,3877.852,3419.269,1404.717,2688.08,-18.68,-20.64,-21.57
3,D18,17.60,2722.68,7.480,227.979,357.008,256.242,404.526,364,178,...,20.606,424.618,63.627,4048.370,3474.219,1427.237,2750.98,-18.91,-20.90,-21.84
4,D18-Cl,16.74,2788.48,7.661,229.756,356.252,259.923,402.943,364,178,...,20.805,432.840,65.462,4285.326,3543.854,1455.776,2787.54,-19.07,-21.08,-22.03


In [ ]:
# 将预处理完成的数据导出
polymerdata.to_excel("polymer-dragon-afterprogress.xlsx")